In [53]:
from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer
from numpy import dot
from numpy.linalg import norm
import numpy as np
from nltk.corpus import stopwords
from pymongo import MongoClient
import pandas as pd
from math import log

In [41]:
uri = 'mongodb://localhost:27017'
client = MongoClient(uri)
db = client['news_db']
collection = db['article']
articles = []
url_dict = {}
doc_count  = 0
for data in collection.find({}) : 
    if 'tech' in data['url']:
        url_dict[doc_count]= data['url'] 
        articles.append(data['content'])
corpus = [article for article in articles if article != None]
print("Total Article Count : ", len(corpus))
print("Sample : ", corpus[0])
print("Sample : ", type(corpus[0]))

Total Article Count :  50
Sample :   CNN — With Christmas just days away, shoppers have a shrinking window to get holiday shopping done and shipping started. A wide range of unanticipated problems might delay shipping around the holidays: surging delivery volume, inclement weather, out of stock items and ground shipping risks. “At this point I don’t typically recommend that people do ground shipping and take their chances,” shopping expert Trae Bodge told CNN. “If a retailer is shipping from that same state, you may think it may arrive in a day, but, because of the volume of packages being shipped around the country, I just worry that you might encounter bottlenecks and delays. I would be very mindful.” Despite the potential issues, procrastinators can still find a way to get their gifts on time. “It’s not doomsday if you’ve waited until the last minute,” Bodge said. “There are many options.” For those still wrapping up their holiday shopping, here are the still-upcoming final shipping

In [59]:

stop_words = stopwords.words('english')
vector = TfidfVectorizer(stop_words=stop_words).fit(corpus)

result= vector.transform(corpus).toarray()
print(vector.transform(corpus).toarray())
print(vector.vocabulary_)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.0360461  0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.02498757 ... 0.         0.         0.        ]]
{'cnn': 1133, 'christmas': 1077, 'days': 1504, 'away': 610, 'shoppers': 4906, 'shrinking': 4925, 'window': 5968, 'get': 2437, 'holiday': 2639, 'shopping': 4907, 'done': 1742, 'shipping': 4903, 'started': 5143, 'wide': 5952, 'range': 4355, 'unanticipated': 5655, 'problems': 4205, 'might': 3480, 'delay': 1553, 'around': 516, 'holidays': 2640, 'surging': 5305, 'delivery': 1565, 'volume': 5850, 'inclement': 2779, 'weather': 5917, 'stock': 5179, 'items': 2966, 'ground': 2509, 'risks': 4646, 'point': 4070, 'typically': 5638, 'recommend': 4409, 'people': 39

In [ ]:
for index , tfidf_list in enumerate(result): # index는 문서 id   , tfdif_list 는 각 단어별 tf-idf 수치를 기록한 리스트 

    for word_index , val in enumerate(tfidf_list):
        if val != 0 :
            word =  [key for key in vector.vocabulary_.keys()  if vector.vocabulary_[key] == word_index]
            if len(word) >0:
                word = word[0]
            print( " Word  :  ",  word, " TF-IDF : ", val  , "Real News : ", corpus[index])
            



In [55]:
# Cosine Similarity를 이용하여 유사한 뉴스 기사 Select 해보기
print("Total Vector Count : ", len(result))

def get_cosine_similarity(vector1, vector2 ):
    np_vector1 = np.array(vector1)
    np_vector2 =np.array(vector2)
    return  dot(np_vector1 , np_vector2) / (norm(np_vector1)*norm(np_vector2))
    

for index1 , tfidf_vector1 in enumerate(result):
    for index2 , tfidf_vector2 in enumerate(result):
                if index1 == index2  :
                    continue
                else:
                    similarity = get_cosine_similarity(tfidf_vector1 , tfidf_vector2)
                    if similarity > 0.1 :
                        print(f"{index1} <-> {index2}  ===  {similarity}")
                        




Total Vector Count :  50
1 <-> 5  ===  0.15050655594963078
2 <-> 4  ===  0.11085032286692414
4 <-> 2  ===  0.11085032286692414
5 <-> 1  ===  0.15050655594963078
6 <-> 30  ===  0.11225383609925002
6 <-> 43  ===  0.13327081222680726
7 <-> 30  ===  0.10087123290266146
7 <-> 43  ===  0.1562350018840026
8 <-> 16  ===  0.11036828650952747
9 <-> 28  ===  0.1071363417515094
10 <-> 12  ===  0.21511026125956095
10 <-> 19  ===  0.2575539836836891
10 <-> 22  ===  0.2362717168616818
11 <-> 14  ===  0.1959380577802459
11 <-> 21  ===  0.12291270288591234
11 <-> 44  ===  0.10688465098062445
11 <-> 45  ===  0.14181180635207713
12 <-> 10  ===  0.21511026125956095
12 <-> 13  ===  0.11061607623955558
12 <-> 19  ===  0.16415465856970773
12 <-> 22  ===  0.21653586993811408
13 <-> 12  ===  0.11061607623955558
13 <-> 22  ===  0.1061499398714329
13 <-> 32  ===  0.10327134828906485
14 <-> 11  ===  0.1959380577802459
14 <-> 16  ===  0.1055896503667761
14 <-> 21  ===  0.11909061414935984
14 <-> 26  ===  0.1006268

In [60]:
print(len(corpus))
print(corpus[12])
print(corpus[10])
for data in collection.find({}) : 
    if data['content'] == corpus[22] :
        print(data['url'])

50
 CNN — A complex but concerning method of gaining control over a user’s iPhone and permanently locking them out the device appears to be on the rise. Some iPhone thieves are exploiting a security setting, called the recovery key, that makes it nearly impossible for owners to access their photos, messages, data and more, according to a recent Wall Street Journal report. Some victims also told the publication their bank accounts were drained after the thieves gained access to their financial apps. It’s important to note, however, this type of takeover is hard to pull off. It requires a criminal essentially watching an iPhone user enter the device’s passcode – for example, by looking over their shoulder at a bar or sporting event – or manipulating the device’s owner so they’ll share their passcode. And that’s all before they physically steal the device. From there, a thief could use the passcode to change the device’s Apple ID, turn off “Find my iPhone” so their location can’t be track